In [33]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import datetime
import urllib

# Create ChromeOptions object
chrome_options = Options()

driver_path = ChromeDriverManager().install()
driver = webdriver.Chrome(options=chrome_options)

driver.get("https://www.smu.ac.kr/kor/life/restaurantView4.do")
driver.find_element(By.CSS_SELECTOR, "#ko > div.board-name-thumb.board-wrap > ul > li:nth-child(1) > dl > dt > a").click() # 이번주 들어가는 것만
time.sleep(0.5)

# 새로 열린 페이지로 전환 
driver.switch_to.window(driver.window_handles[-1])

# 현재 url 가져오기
final_url = driver.current_url
driver.quit()
print(final_url)
#------------------------------------------------------------------------------현재 url// 이전에는 웹페이지 접속경로임

response = requests.get(final_url)
soup = BeautifulSoup(response.text, "html.parser")

image_tag = soup.select_one(".fr-view img")
image_src = image_tag.get("src")
img_url = "https://www.smu.ac.kr" + image_src

now = datetime.datetime.now()
image_path = f"학생식당/" + now.strftime("%m%d") + ".png"

urllib.request.urlretrieve(img_url, image_path) 

https://www.smu.ac.kr/kor/life/restaurantView4.do?mode=view&articleNo=741356&article.offset=0&articleLimit=10


('학생식당/1203.png', <http.client.HTTPMessage at 0x130126bd0>)

In [41]:
import cv2
import os
# from google.colab.patches import cv2_imshow
from datetime import datetime

# Read the image
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding or any other preprocessing steps if needed
# For example, you can use cv2.threshold
_, thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

# Find contours
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a directory to save the cropped images
output_dir_cropped = f"학생식당/cropped_images_student1203"
os.makedirs(output_dir_cropped, exist_ok=True)

# Loop through each contour
for i, cnt in enumerate(contours):
    x, y, w, h = cv2.boundingRect(cnt)

    # Check the specified conditions
    if h >= 100 and 100 <= w < 800:
        # Crop the region of interest (ROI)
        cropped_image = image[y:y+h, x:x+w]

        # Save the cropped image
        relative_output_path = os.path.join(output_dir_cropped, f"student{i}.png")
        cv2.imwrite(relative_output_path, cropped_image)

